# **[인공지능] 과제4 Cliff Walking 예제 구현**
*   **QLearning Class 및 Sarsa Class를 완성하여 결과를 살펴보는 것이 목표**입니다.
*   기본적인 코드는 아래 노트에 모두 작성되어 있습니다. 비어있는 함수 부분을 완성하면 됩니다.
*   **과제 수행 시 주의사항: 외부 라이브러리로 Q-learning 및 Sarsa 적용하지 말 것, 수업 때 배운 내용대로 Q-learning과 Sarsa를 주어진 함수에 구현할 것.** 웹 상에 있는 다양한 Q-learning 및 Sarsa 코드를 참고하는 것은 괜찮습니다.
*   **보고서 작성 내용**: 여러분이 완성한 Q-learning 및 sarsa 알고리즘의 내용과 결과의 의미를 분석하는 내용을 작성하면 됩니다.
작성한 코드와 실행 결과를 첨부하길 바라며, 코드에는 자세한 주석을 필수적으로 포함하기 바랍니다. 보고서는 PDF로 제출바랍니다.
*   보고서는 12월 13일 오후 11시 59분까지 블랙보드에 보고서 형태로 제출하면 됩니다. 지각은 0점입니다.

# **본 노트를 본인의 drive로 복사하여 활용하기 바랍니다.**

In [ ]:
!pip install pygame


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.0 MB/s 


In [ ]:
import numpy as np
import random
from tqdm import tqdm
from collections import defaultdict, namedtuple, deque
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv # Cliff Walking 환경

#추가한것
import pandas as pd
import gym

#####
import torch
import torch.nn as nn
import torch.optim as optim

본 과제는 OpenAI Gym 환경에 기반하여 작성되었습니다. Gym 라이브러리는 학습을 적용할 수 있는 다양한 환경을 제공합니다. 여기서는 수업에서 다뤘던 Cliff Walking 환경을 활용합니다.

Cliff Walking 예제의 state 개수는 48개, action 개수는 4개입니다.

In [ ]:
env = CliffWalkingEnv()
print ('Number of states: ', env.nS)
print ('Number of actions :', env.nA)

Number of states:  48
Number of actions : 4


printPolicy 함수는 학습된 Q-value를 입력하면 해당하는 Q-value의 greedy 정책이 출력되도록 합니다. Q-learning 및 Sarsa를 이용하여 학습된 정책을 출력하기 위해 필요합니다.

* 함수의 입력값은 'Q[state][action]'의 형태로 정리된 행동가치 함수값 (48,4) 행렬이고, 입력에 따른 greedy 정책을 출력합니다.

**구현되어있는 함수를 그대로 활용하면 됩니다.**

In [ ]:
def printPolicy(self, Q):
    action = ['↑', '→', '↓', '←', '×']
    policy = np.array([np.argmax(Q[key]) 
                      if key in Q else -1 
                      for key in np.arange(48)])
    v = ([np.max(Q[key]) if key in Q else 0 for key in np.arange(48)])
    actions = np.stack([action for _ in range(len(policy))], axis=0)

    print(np.take(actions, np.reshape(policy, (4, 12))))
    print('')

argmax는 최댓값을 반환한다. 
np.arange(시작점(생략 시 0), 끝점(미포함), step size(생략 시 1)) 인 array 생성 즉 0부터 47까지인 배열 생성하고 key 를 안에 넣어서 찾는다.


즉 0부터 

Q-learning과 Sarsa 모두 behavior policy로 epsilon-greedy를 쓴다고 가정합니다.

아래는 Q-learning 알고리즘을 수행하는 Class의 정의입니다.
*   `update()` 메쏘드의 경우 state, action, reward, next_state, next_action이 주어졌을 때 Q-value를 업데이트하는 함수입니다.
*   `act()` 메쏘드의 경우 $\epsilon$-greedy 정책에 따라 action을 선택하는 함수입니다.

In [ ]:
class QLearning:
    def __init__(self):
        self.action_no = 4
        self.alpha = 0.01
        self.gamma = 0.99
        self.epsilon = 0.5
        self.q_values = defaultdict(lambda: [0.0] * self.action_no)

    def update(self, state, action, reward, next_state, next_action):
        max_act = np.argmax(self.q_values[next_state])
        # argmax는 최댓값을 반환한다. 
        old = self.q_values[state][action] # 현재의 상태는 old로 저장한다.
        new = self.q_values[next_state][max_act] # 
        self.q_values[state][action] = (1-self.alpha) * old+ self.alpha * ( self.gamma * new+ reward)
        #현재상태 (S, state), 현재행동 (A, action), 보상 (R, reward),
        #  다음상태 (S', next_state), 다음행동 (A', next_action)
        # 위 다섯가지 원소를 가지고 시간차(TD)를 학습합니다.
        # SARSA와 다른점이 있다면 SARSA가 정책 반복의 샘플 기반 구현법이라면,
        #Q러닝은 가치 반복의 샘플 기반 구현 기법이다.
    def act(self, state):
        if random.random() < self.epsilon:
            return random.randrange(self.action_no)
        else:
            return np.argmax(self.q_values[state])
      #상태정보를 넣으면 행동을 반환한다.

아래는 Sarsa 알고리즘을 수행하는 Class의 정의입니다.
*   `update()` 메쏘드의 경우 state, action, reward, next_state, next_action이 주어졌을 때 Q-value를 업데이트하는 함수입니다.
*   `act()` 메쏘드의 경우 $\epsilon$-greedy 정책에 따라 action을 선택하는 함수입니다.



In [ ]:
class Sarsa:
    def __init__(self):
        self.action_no = 4
        self.alpha = 0.01
        self.gamma = 0.99
        self.epsilon = 0.5
        self.q_values = defaultdict(lambda: [0.0] * self.action_no)

    def update(self, state, action, reward, next_state, next_action):
        new = self.q_values[next_state][next_action]
        now = self.q_values[state][action]
        self.q_values[state][action] =  self.alpha * (reward + self.gamma * new)+ (1 - self.alpha) * now 
        #업데이트 공식에 맞추어서 현재의 행동 큐함수와
        #다음 행동 큐함수를 구하고 이를 바탕으로 현재의 행동 큐함수의 업데이트를 진행한한다.
    def act(self, state):
        if random.random() < self.epsilon:
            return random.randrange(self.action_no)
            #randrange()는 랜덤으로 무작위하게 숫자를 반환하는 함수이다. 
            #randrange(self.action_no)는 0 <= x < self.action_no 의 범위 내에서의 랜덤한 정수(int)를 반환합니다. 
        else:
            return np.argmax(self.q_values[state])
            #ϵ - 탐욕 정책에 따라 0에서 1사이의 실수 난수를 뽑아서 
            #그 값이 epsilon 보다 작으면 행동중에서 무작위로 하나를
            #뽑아 행동하고 아닐 경우 
            #큐함수 테이블에서 해당 상태의 큐함수 리스트 중에서 가장 큰 값에 따라 행동한한다
    # 상태정보를 넣으면 행동을 반환한다. 

Q-learning 및 Sarsa Class를 정의하고 5000 episode 동안 학습을 수행합니다.

Gym 라이브러리의 환경에서는 `step(action)` 메쏘드를 통해 해당하는 time-step에서 action을 수행한 효과를 얻을 수 있습니다. 해당 메쏘드에서는 action을 수행하여 얻어지는 보상 (reward), 다음 상태 (next_state), done (episode 종료여부) 등이 출력으로 주어집니다.

In [ ]:
agent_QL = QLearning()
for ep in range(5000): # 5000번의 에피소드를 통해 학습합니다.
    done = False
    state = env.reset()
    action = agent_QL.act(state)
    
    ep_rewards = 0
    while not done:
        next_state, reward, done, _, _ = env.step(action)
        # _은 값은 반환하지만 쓰지는 않겠다는 의미이다. 
        next_act = agent_QL.act(next_state)
        # 다음 상태가 들어간 액션을 next action에 대입한다.
        agent_QL.update(state, action, reward, next_state, next_act)
        #업데이트를 해준다. 
        state = next_state
        #다음 상태를 대입한다.
        action = next_act
        # 액션도 다음 액션을 넣어줌으로써 다음 행동을 준비한다.
        ep_rewards += reward # 보상로 새로 update한다. 

In [ ]:
agent_Sa = Sarsa()
for ep in range(5000):
    done = False
    state = env.reset()
    action = agent_Sa.act(state)
    
    ep_rewards = 0
    while not done:
        next_state, reward, done, _, _ = env.step(action)
        # _은 값은 반환하지만 쓰지는 않겠다는 의미이다. 
        next_action = agent_Sa.act(next_state)
        # 다음 상태가 들어간 액션을 next action에 대입한다
        agent_Sa.update(state, action, reward, next_state, next_action)
         #업데이트를 해준다. 
        state = next_state
         #다음 상태를 대입한다.
        action = next_action
         # 액션도 다음 액션을 넣어줌으로써 다음 행동을 준비한다.
        ep_rewards += reward
        # reward를 보상해준다. 

학습된 Q-value를 이용하여 학습된 정책을 출력합니다.

In [ ]:
print('Learned policy by Q-learning')
printPolicy(agent_QL ,agent_QL.q_values)
print('Learned policy by Sarsa')
printPolicy(agent_Sa,agent_Sa.q_values)

Learned policy by Q-learning
[['→' '→' '→' '→' '↓' '→' '→' '→' '→' '→' '↓' '↓']
 ['→' '→' '→' '→' '↓' '↓' '↓' '→' '↓' '↓' '↓' '↓']
 ['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['↑' '×' '×' '×' '×' '×' '×' '×' '×' '×' '×' '↑']]

Learned policy by Sarsa
[['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['↑' '→' '↑' '↑' '↑' '↑' '→' '→' '→' '→' '→' '↓']
 ['↑' '↑' '↑' '↑' '↑' '↑' '↑' '→' '→' '↑' '→' '↓']
 ['↑' '×' '×' '×' '×' '×' '×' '×' '×' '×' '×' '↑']]



In [ ]:
env.close()